In [32]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.scenario import Trigger


# Accessing the DSS Global/Project Variables
proj = dataiku.Project()
variables = proj.get_variables()

In [33]:
ntile10_dataset = dataiku.Dataset("dpc_covid19_prepared_windows_by_ntile10")
dpc_dataset = dataiku.Dataset("dpc_covid19_prepared")

In [34]:
ntile10_df = ntile10_dataset.get_dataframe()
dpc_df = dpc_dataset.get_dataframe()

/Users/terrijohn/Library/dss/design/code-envs/python/python37_holidays/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [35]:
ntile10_df.head()

,ntile10,nuovi_positivi_min,nuovi_positivi_max,nuovi_positivi_avg,nuovi_positivi_sum,nuovi_positivi_stddev,count
0,1,78,451,253.000000,21252,84.220022,84
1,2,463,1397,918.452381,77150,294.270362,84
2,3,1400,2818,2035.096386,168913,456.517374,83
3,4,2834,4717,3713.726190,311953,554.433464,84
4,5,4720,7537,5963.337349,494957,817.652847,83


In [36]:
dpc_df.head()

,data,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,is_holiday,is_weekend
0,2020-02-24 18:00:00+00:00,ITA,101,26,127,94,221,0,221,1,7,NaN,NaN,229,4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,2020-02-25 18:00:00+00:00,ITA,114,35,150,162,311,90,93,1,10,NaN,NaN,322,8623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,2020-02-26 18:00:00+00:00,ITA,128,36,164,221,385,74,78,3,12,NaN,NaN,400,9587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,2020-02-27 18:00:00+00:00,ITA,248,56,304,284,588,203,250,45,17,NaN,NaN,650,12014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,2020-02-28 18:00:00+00:00,ITA,345,64,409,412,821,233,238,46,21,NaN,NaN,888,15695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [37]:
#Set variables for ntile ranges in order to compare to other dataset 
for num in range(1,11):
    variables["standard"][f'n{num}'] = range(int(df['nuovi_positivi_min'][df['ntile10'] == num]), int(df['nuovi_positivi_max'][df['ntile10'] == num]))

In [38]:
variables["standard"]

{'url': 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv',
 'csv_name': 'dpc_covid19_ita_andamento_nazionale_df',
 'min_new_daily_pos': 74,
 'max_new_daily_pos': 220532,
 'n1': range(78, 451),
 'n2': range(463, 1397),
 'n3': range(1400, 2818),
 'n4': range(2834, 4717),
 'n5': range(4720, 7537),
 'n6': range(7548, 13318),
 'n7': range(13331, 18236),
 'n8': range(18255, 27214),
 'n9': range(27354, 58861),
 'n10': range(59555, 220532)}

In [25]:
# 79 in variables["standard"]['n1']

True

In [41]:
for row in dpc_df['nuovi_positivi']:
    for ntile in range(1,11):
        if row in variables["standard"][f'n{ntile}']:
            print(row, ntile)

221
93
78
250
238
240
566
342
466
587
769
778
1247
1492
1797
977
2313
2651
2547
3497
3590
3233
3526
4207
5322
5986
6557
5560
4789
5249
5210
6153
5959
5974
5217
4050
4053
4782
4668
4585
4805
4316
3599
3039
3836
4204
3951
4694
4092
3153
2972
2667
3786
3493
3491
3047
2256
2729
3370
2646
3021
2357
2324
1739
2091
2086
1872
1965
1900
1389
1221
1075
1444
1401
1327
1083
802
744
1402
888
992
789
875
675
451
813
665
642
652
669
531
300
397
584
593
516
416
355
178
318
321
177
518
270
197
280
283
202
379
163
346
338
303
210
329
333
251
262
224
218
122
190
296
259
175
174
126
142
187
201
223
235
192
208
138
193
229
276
188
234
169
114
163
230
233
249
219
190
129
282
306
252
275
255
170
212
289
386
379
295
239
159
190
384
402
552
347
463
259
412
481
523
574
629
479
320
403
642
845
947
1071
1210
953
878
1367
1411
1462
1444
1365
996
978
1326
1397
1733
1694
1297
1108
1370
1434
1597
1616
1501
1458
1008
1229
1452
1585
1907
1638
1587
1350
1392
1640
1786
1912
1869
1766
1494
1648
1851
2548
2499
2844
2578
22